In [1]:
import os
import time
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import FinNews as fn
import re

# Dash modules
import dash
import dash_table
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go

# Set up jupyter proxy
# JupyterDash.infer_jupyter_proxy_config() 

%matplotlib inline

In [2]:
# Getting all file paths

path = r'assets/historical-symbols' # use your path
all_files = glob.glob(path + "/*.csv")

# Creating list to append all ticker dfs to
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Concat all ticker dfs
stock_df = pd.concat(li, axis=0, ignore_index=True)

stock_df['Date'] = pd.to_datetime(stock_df['Date'])

# Creating Moving Average Technical Indicator
# Using this aritcle https://towardsdatascience.com/building-a-comprehensive-set-of-technical-indicators-in-python-for-quantitative-trading-8d98751b5fb

stock_df['SMA_5'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window = 5).mean())
stock_df['SMA_15'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window = 15).mean())
stock_df['SMA_ratio'] = stock_df['SMA_15'] / stock_df['SMA_5']

# Bollinger bands
stock_df['15MA'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window=15).mean())
stock_df['SD'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window=15).std())
stock_df['upperband'] = stock_df['15MA'] + 2*stock_df['SD']
stock_df['lowerband'] = stock_df['15MA'] - 2*stock_df['SD']

available_indicators1 = list(stock_df['ticker'].unique())

In [4]:
title_style = {'display': 'inline-block',
               'textAlign':'left',
               'verticalAlign':'center',
               'lineHeight':2.5,
               'height':75,
               'border': 'thick black solid',\
               'width':'85%',
               'backgroundColor': 'rgb(212, 150, 18)'}

title_link_style = {'display': 'inline-block',
               'textAlign':'center',
               'vertical-align':'center',
               'lineHeight':5,
               'height':75,
               'width':'10%',
               'float':'right',
               'border': 'thick black solid',\
               'fontSize':12,
               'backgroundColor': 'rgb(212, 150, 18)'}


tab_style = {'display': 'inline-block',
             'textAlign':'center',
             'verticalAlign':'top',
             'width':'15%',
             'border': 'thin lightgrey solid',
             'height':20}

blank_tab_style = {'display': 'inline-block',
                   'width':'15%',
                   'textAlign':'center',
                   'verticalAlign':'top',
                   'float':'right',
                   'border': 'thin lightgrey solid',
                   'fontSize':10,
                   'height':20}

ticker_style = {'display': 'inline-block',
                'textAlign':'center',
                'vertical-align':'top',
                'width':'20%',
                'float':'right',
                'border':'thin lightgrey solid',
                'height':300}

chart_style = {'display':'inline-block',
               'textAlign':'center',
               'vertical-align':'top',
               'width':'60%',
               'float':'middle',
               'textAlign':'center',
               'border':'thin lightgrey solid',
               'height':300}

portfolio_style = {'display': 'inline-block',
                'textAlign':'center',
                'vertical-align':'top',
                'width':'20%',
                'float':'left',
                'border':'thin lightgrey solid',
                'height':300}

chart_style_b = {'display':'inline-block',
               'textAlign':'center',
               'vertical-align':'top',
               'width':'60%',
               'float':'middle',
               'textAlign':'center',
               'border':'thin lightgrey solid',
               'height':200}

portfolio_style_b = {'display': 'inline-block',
                'textAlign':'left',
                'fontSize':12,
                'vertical-align':'top',
                'width':'20%',
                'float':'left',
                'border':'thin lightgrey solid',
                'height':200}

news_style = {'fontSize':11,
              'color':'white',
              'textAlign':'left',
              'height':'auto',
#               'height':27,
              'margin':3,
              "lineHeight":1.5,
              'float':'middle'}

news_style_b = {'display': 'inline-block',
                'textAlign':'left',
                'vertical-align':'top',
                'width':'20%',
                'float':'right',
                'border':'thin lightgrey solid',
                'height':300,'margin':0,'overflowY':'scroll',
                'color':'white',
                'backgroundColor':'black',
                'borderRadius': '.4rem'}



#                'vertical-align':'middle',

In [5]:
app = JupyterDash(__name__,suppress_callback_exceptions=True)

server = app.server

app.layout = html.Div([

    html.Div([
        html.H1('Dashboard Title',style=title_style),
        html.H3('Dashboard Info Link',style=title_link_style)
        ]),

    dcc.Tabs(id='tabs-example', value='tab-1', children=[
        dcc.Tab(label='Portfolio Performance', value='tab-1'),
        dcc.Tab(label='Model Performance', value='tab-2'),
        dcc.Tab(label='Tab three', value='tab-3')
    ],style={'width':'50%','height':50}),
    html.Div(id='tabs-example-content')
])

@app.callback(dash.dependencies.Output('tabs-example-content', 'children'),
              [dash.dependencies.Input('tabs-example', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            
                html.Div([
                    dcc.Dropdown(id='ticker',
                    options=[{'label': i, 'value': i} for i in available_indicators1],
                    value='CSCO') # the default is code_module AAA
                    ]),
                
                html.Div([
                    html.H2('Portfolio Performance',style=portfolio_style),
                    dcc.Graph(id='price_chart',style=chart_style),
                    html.Div(id='news_list',children=news_info,style=news_style_b)
                    ]),
                    
                html.Div([
                    html.H2('Other Portfolio Statistics',style=portfolio_style_b),
                    html.H2('News Info',style=chart_style_b),
                    ],style={'height':200})
    
                ])
                
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab content 2')
        ])
    
# Callback to connect input(s) to output(s)
@app.callback(dash.dependencies.Output('price_chart','figure'),
    [dash.dependencies.Input('ticker','value')])

# Step 3: Define the graph with plotly express
def update_ticker(ticker):
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=stock_df[stock_df['ticker']==ticker]['Date'],
                             y=stock_df[stock_df['ticker']==ticker]['Close'],
                            line={"color": "#228B22"},
                            mode="lines"))

    fig.update_layout(title_text=f'{ticker} Closing Price',title_x=0.5,
                         template="plotly_dark",font=dict(size=10),xaxis_showgrid=False,
                         yaxis_title="Closing Price",margin={"r": 20, "t": 35, "l": 20, "b": 10})

    return fig

# Creating callback to get news when ticker changes
@app.callback(dash.dependencies.Output('news_list', 'children'),
              [dash.dependencies.Input('ticker', 'value')])
def update_news(ticker):
    
    seeking_alpha = fn.SeekingAlpha(topics=['$'+ticker], save_feeds=True)
    
    news = seeking_alpha.get_news()
    
#     title_strings = [re.sub('[^A-Za-z0-9,\s]+', '', item['title']).lower().replace(" ", "-") for item in news[:5]]
#     url_ids = [re.sub('MarketCurrent:','news/',item) for item in news[:5]]
    
    news_info = html.Div(f'News for {ticker}',style={'backgroundColor':'gray'}),\
                    dbc.ListGroup([
                        dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[0]['title'],style=news_style),\
                                        href=(news[0]['link'])),\
                                        html.A(html.P(news[0]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[1]['title'],style=news_style),\
                                        href=(news[1]['link'])),\
                                        html.A(html.P(news[1]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[2]['title'],style=news_style),\
                                        href=(news[2]['link'])),\
                                        html.A(html.P(news[2]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[3]['title'],style=news_style),\
                                        href=(news[3]['link'])),\
                                        html.A(html.P(news[3]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[4]['title'],style=news_style),\
                                        href=(news[4]['link'])),\
                                        html.A(html.P(news[4]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    ],flush=True)
                    
    return news_info

if __name__ == '__main__':
    app.run_server(debug=True,port=8055)

Dash app running on http://127.0.0.1:8055/


### Notes
1. How do I set a relative height
2. Could add ticker - https://community.plotly.com/t/strip-ticker-label/48348/6


In [20]:
html.Div([
        # Adding drop down to filter by ticker
        dcc.Dropdown(id='ticker',
            options=[{'label': i, 'value': i} for i in list(stock_df['ticker'].unique())],
            value='CSCO',style={'margin':'5px','width':'40%'}), # the default is code_module AAA

        dcc.Dropdown(id='industry_ticker',
            options=[{'label': i, 'value': i} for i in list(stock_df['sector'].unique())],
            value='Technology',style={'margin':'5px','width':'40%'}) # the default is code_module AAA
            ])

Div([Dropdown(id='ticker', options=[{'label': 'CSCO', 'value': 'CSCO'}, {'label': 'UAL', 'value': 'UAL'}, {'label': 'TROW', 'value': 'TROW'}, {'label': 'ISRG', 'value': 'ISRG'}, {'label': 'NVR', 'value': 'NVR'}, {'label': 'PRGO', 'value': 'PRGO'}, {'label': 'TPR', 'value': 'TPR'}, {'label': 'DVN', 'value': 'DVN'}, {'label': 'CE', 'value': 'CE'}, {'label': 'MRO', 'value': 'MRO'}, {'label': 'BA', 'value': 'BA'}, {'label': 'VRTX', 'value': 'VRTX'}, {'label': 'GILD', 'value': 'GILD'}, {'label': 'NLSN', 'value': 'NLSN'}, {'label': 'EQIX', 'value': 'EQIX'}, {'label': 'TER', 'value': 'TER'}, {'label': 'MDT', 'value': 'MDT'}, {'label': 'V', 'value': 'V'}, {'label': 'QRVO', 'value': 'QRVO'}, {'label': 'A', 'value': 'A'}, {'label': 'FOX', 'value': 'FOX'}, {'label': 'FLT', 'value': 'FLT'}, {'label': 'MO', 'value': 'MO'}, {'label': 'SWKS', 'value': 'SWKS'}, {'label': 'ENPH', 'value': 'ENPH'}, {'label': 'MCHP', 'value': 'MCHP'}, {'label': 'CDNS', 'value': 'CDNS'}, {'label': 'WLTW', 'value': 'WLTW'}

In [22]:
[{'label': i, 'value': i} for i in ['Stocks','Sector']]

[{'label': 'Stocks', 'value': 'Stocks'},
 {'label': 'Sector', 'value': 'Sector'}]

In [6]:
stock_df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,sector,ticker,SMA_5,SMA_15,SMA_ratio,15MA,SD,upperband,lowerband
0,1985-07-22,0.0,4.12,3.84,4.12,59400.0,0.0,0.0,Energy,DVN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985-07-23,0.0,4.26,4.07,4.12,20800.0,0.0,0.0,Energy,DVN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985-07-24,0.0,4.07,3.94,4.03,32600.0,0.0,0.0,Energy,DVN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985-07-25,0.0,4.07,4.03,4.07,16600.0,0.0,0.0,Energy,DVN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1985-07-26,0.0,4.07,3.98,4.03,10400.0,0.0,0.0,Energy,DVN,4.074,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=stock_df[stock_df['ticker']=='AAPL']['Date'],
                         y=stock_df[stock_df['ticker']=='AAPL']['Close'],
                        line={"color": "#228B22"},
                        mode="lines"))

fig.update_layout(title_text='AAPL Closing Price',title_x=0.5,
                     template="plotly_dark",font=dict(size=10),xaxis_showgrid=False,
                     yaxis_title="Closing Price",margin={"r": 20, "t": 35, "l": 20, "b": 10})

In [72]:
df = df.set_index('Date')
df

,Close,Dividends,High,Low,Open,Stock Splits,Volume,sector,test,ticker,SMA_5,SMA_15,SMA_ratio,15MA,SD,upperband,lowerband
Date,,,,,,,,,,,,,,,,,
1980-12-12,0.100751,0.0,0.101189,0.100751,0.100751,0.0,469033600.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-15,0.095495,0.0,0.095933,0.095495,0.095933,0.0,175884800.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-16,0.088485,0.0,0.088923,0.088485,0.088923,0.0,105728000.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-17,0.090676,0.0,0.091114,0.090676,0.090676,0.0,86441600.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-18,0.093304,0.0,0.093742,0.093304,0.093304,0.0,73449600.0,Technology,NaN,AAPL,0.093742,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-16,130.149994,0.0,130.889999,128.460007,130.369995,0.0,91815000.0,Technology,NaN,AAPL,128.745998,126.600665,0.983337,126.600665,2.102349,130.805364,122.395967
2021-06-17,131.789993,0.0,132.550003,129.649994,129.800003,0.0,96721700.0,Technology,NaN,AAPL,129.881996,126.929998,0.977272,126.929998,2.494541,131.919080,121.940917
2021-06-18,130.460007,0.0,131.509995,130.240005,130.710007,0.0,108787300.0,Technology,NaN,AAPL,130.503998,127.275332,0.975260,127.275332,2.605871,132.487074,122.063590


In [8]:
stock_df.ticker.unique()

array(['DVN', 'TER', 'FLT', 'SWKS', 'MSCI', 'WBA', 'DTE', 'DISH', 'NOV',
       'PEG', 'REG', 'LOW', 'CERN', 'NOC', 'BXP', 'ALLE', 'PENN', 'DPZ',
       'CMA', 'PWR', 'HOLX', 'WDC', 'LEG', 'EMR', 'WEC', 'SHW', 'AMAT',
       'NDAQ', 'HES', 'XOM', 'BEN', 'DISCK', 'PTC', 'RF', 'MAR', 'PEAK',
       'EL', 'ALL', 'PAYX', 'AAL', 'MS', 'APA', 'CSX', 'ZION', 'SLB',
       'DLTR', 'WYNN', 'PKG', 'BRK-B', 'PGR', 'BLK'], dtype=object)

In [6]:
fig = go.Figure()

df = stock_df[stock_df['ticker']=='ALL']
df = df.set_index('Date')
    
data = [go.Scatter(x=df.index,
                         y=df['Close'],
                        line={"color": "#228B22"},
                        mode="lines")]

layout = dict(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig = go.FigureWidget(data=data, layout=layout)

def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 5, in_view.High.max() + 5]

fig.layout.on_change(zoom, 'xaxis.range')


fig


FigureWidget({
    'data': [{'line': {'color': '#228B22'},
              'mode': 'lines',
              'type'…

In [55]:
fig2 = go.Figure()

data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])]

layout = dict(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])            
        ),
        rangeslider=dict(
            visible=False
        ),
        type="date",
    ),
    showlegend=False
)

fig2 = go.FigureWidget(data=data, layout=layout)

def zoom(layout, xrange):
    in_view = df.loc[fig2.layout.xaxis.range[0]:fig2.layout.xaxis.range[1]]
    fig2.layout.yaxis.range = [in_view.Low.min() - 1, in_view.High.max() + 1]

fig2.layout.on_change(zoom, 'xaxis.range')


fig2

FigureWidget({
    'data': [{'close': array([  7.49,   7.3 ,   7.18, ..., 129.75, 129.57, 130.44]),
          …

In [110]:
fig.layout.xaxis.range

('1980-12-12', '2021-06-22')

In [114]:
df.loc['2020-12-11'].High

122.3689794627517

In [78]:
df

,Close,Date,Dividends,High,Low,Open,Stock Splits,Volume,sector,test,ticker,SMA_5,SMA_15,SMA_ratio,15MA,SD,upperband,lowerband
3559733,0.100751,1980-12-12,0.0,0.101189,0.100751,0.100751,0.0,469033600.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3559734,0.095495,1980-12-15,0.0,0.095933,0.095495,0.095933,0.0,175884800.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3559735,0.088485,1980-12-16,0.0,0.088923,0.088485,0.088923,0.0,105728000.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3559736,0.090676,1980-12-17,0.0,0.091114,0.090676,0.090676,0.0,86441600.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3559737,0.093304,1980-12-18,0.0,0.093742,0.093304,0.093304,0.0,73449600.0,Technology,NaN,AAPL,0.093742,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3569946,130.149994,2021-06-16,0.0,130.889999,128.460007,130.369995,0.0,91815000.0,Technology,NaN,AAPL,128.745998,126.600665,0.983337,126.600665,2.102349,130.805364,122.395967
3569947,131.789993,2021-06-17,0.0,132.550003,129.649994,129.800003,0.0,96721700.0,Technology,NaN,AAPL,129.881996,126.929998,0.977272,126.929998,2.494541,131.919080,121.940917
3569948,130.460007,2021-06-18,0.0,131.509995,130.240005,130.710007,0.0,108787300.0,Technology,NaN,AAPL,130.503998,127.275332,0.975260,127.275332,2.605871,132.487074,122.063590
3569949,132.300003,2021-06-21,0.0,132.410004,129.210007,130.300003,0.0,79576500.0,Technology,NaN,AAPL,130.867999,127.787999,0.976465,127.787999,2.793728,133.375455,122.200543


In [58]:
in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
in_view

,Close,Date,Dividends,High,Low,Open,Stock Splits,Volume,sector,test,ticker,SMA_5,SMA_15,SMA_ratio,15MA,SD,upperband,lowerband


In [37]:
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]

fig.layout.on_change(zoom, 'xaxis.range')

In [38]:
fig

FigureWidget({
    'data': [{'line': {'color': '#228B22'},
              'mode': 'lines',
              'type'…

In [5]:
from plotly.subplots import make_subplots
fig = make_subplots(shared_xaxes=True ,rows=2, cols=1)

fig.add_trace(
    go.Scatter(x=df.index,
        y=df['Close'],
        line={"color": "#228B22"},
        mode="lines"),
    row=1, col=1
)

fig.add_trace(
    go.Candlestick(x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close']),
    row=2, col=1
)

fig.update_layout(layout)

def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.Low.min() - 1, in_view.High.max() + 1]

fig.layout.on_change(zoom, 'xaxis.range')

fig.update_yaxes(range=[100,130])

fig

NameError: name 'layout' is not defined

In [62]:
fig.layout.yaxis.range[0]

TypeError: 'NoneType' object is not subscriptable

In [15]:
df.index.unique().max().date()

datetime.date(2021, 6, 22)

In [6]:
app = JupyterDash(__name__,suppress_callback_exceptions=True,external_stylesheets=[dbc.themes.SUPERHERO])

server = app.server

app.layout = html.Div([
    
    html.Button('Button 1', id='btn-nclicks-1',n_clicks=0),
    html.Button('Button 2', id='btn-nclicks-2',n_clicks=0),
    html.Div(id='container-button-timestamp')
])

@app.callback(
    dash.dependencies.Output('container-button-timestamp', 'children'),
    [dash.dependencies.Input('btn-nclicks-1', 'n_clicks'),
     dash.dependencies.Input('btn-nclicks-2', 'n_clicks')])

def update_output(btn1,btn2):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    
    msg = ''
    
    if 'btn-nclicks-1' in changed_id:
        msg = 'Button 1 was most recently clicked'
    elif 'btn-nclicks-2' in changed_id:
        msg = 'Button 2 was most recently clicked'

    return html.Div(msg)

if __name__ == '__main__':
    app.run_server(debug=True,port=8055)

Dash app running on http://127.0.0.1:8055/


In [10]:
dash.callback_context.triggered

MissingCallbackContextException: dash.callback_context.triggered is only available from a callback!